In [1]:
from analyze import *

In [2]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
checklist = Checklist("../../checklist/checklist.csv", checklist_format=ChecklistFormat.CSV)
repo = Repository("../../data/raw/openja/lightfm_demo/")
prompt_format = EvaluationPromptFormat()

In [3]:
evaluator = PerFileTestEvaluator(llm, prompt_format=prompt_format, repository=repo, checklist=checklist)
response = evaluator.evaluate()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:37<00:00, 12.60s/it]


In [4]:
parser = ResponseParser(response)
parser.get_completeness_score(verbose=True)

Report:
                                                                                             Requirement  \
ID  Title                                                                                                  
2.1 Ensure Data File Loads as Expected                 Ensure that data-loading functions correctly l...   
3.2 Data in the Expected Format                        Verify that the data to be ingested matches th...   
3.5 Check for Duplicate Records in Data                Check for duplicate records in the dataset and...   
4.2 Verify Data Split Proportion                       Check that the data is split into training and...   
5.3 Ensure Model Output Shape Aligns with Expectation  Ensure the shape of the model's output aligns ...   
6.1 Verify Evaluation Metrics Implementation           Verify that the evaluation metrics are correct...   
6.2 Evaluate Model's Performance Against Thresholds    Compute evaluation metrics for both the traini...   

                   

'1.5/7'

In [5]:
parser.export_evaluation_report("report.html", "html", exist_ok=True)

/Users/johnshiu/Desktop/ubc_mds/capstone/test-creation/src/test_creation/report.html /Users/johnshiu/Desktop/ubc_mds/capstone/test-creation/src/test_creation


### Consistency evaluation

In [6]:
from consistency_check import *

In [7]:
checklist = Checklist("../../checklist/checklist.csv", checklist_format=ChecklistFormat.CSV) # FIXME: use newer checklist
repo = Repository("../../data/raw/openja/lightfm_demo/")
prompt_format = EvaluationPromptFormat()

In [8]:
gpt35 = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
gpt4o = ChatOpenAI(model="gpt-4o", temperature=0)
evaluator35 = PerFileTestEvaluator(gpt35, prompt_format=prompt_format, repository=repo, checklist=checklist)
evaluator4o = PerFileTestEvaluator(gpt4o, prompt_format=prompt_format, repository=repo, checklist=checklist)

consist_eval = ConsistencyEvaluator()
consist_eval.evaluate(
    models=[
        {'name': 'gpt-3.5-turbo', 'model': evaluator35},
        {'name': 'gpt-4o', 'model': evaluator4o}
    ],
    num_test_runs=2,
    verbose=True,
)

Model: gpt-3.5-turbo
Test Run No.: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:40<00:00, 13.40s/it]


Test Run No.: 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:34<00:00, 11.54s/it]


Model: gpt-4o
Test Run No.: 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:53<00:00, 17.79s/it]


Test Run No.: 2


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:51<00:00, 17.17s/it]


In [9]:
consist_eval.evaluation_reports

,score,report,model_name,test_no
0,0.285714,ID ...,gpt-3.5-turbo,1
1,0.142857,ID ...,gpt-3.5-turbo,2
2,0.714286,ID ...,gpt-4o,1
3,0.714286,ID ...,gpt-4o,2


In [10]:
consist_eval.evaluation_reports['report'].iloc[0]

,ID,Title,Requirement,is_Satisfied,n_files_tested,Observations,Function References
0,2.1,Ensure Data File Loads as Expected,Ensure that data-loading functions correctly l...,0.0,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...
1,3.2,Data in the Expected Format,Verify that the data to be ingested matches th...,0.0,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...
2,3.5,Check for Duplicate Records in Data,Check for duplicate records in the dataset and...,0.0,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...
3,4.2,Verify Data Split Proportion,Check that the data is split into training and...,0.5,3,[(test_cross_validation.py) The code includes ...,[{'File Path': '../../data/raw/openja/lightfm_...
4,5.3,Ensure Model Output Shape Aligns with Expectation,Ensure the shape of the model's output aligns ...,0.0,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...
5,6.1,Verify Evaluation Metrics Implementation,Verify that the evaluation metrics are correct...,1.0,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...
6,6.2,Evaluate Model's Performance Against Thresholds,Compute evaluation metrics for both the traini...,0.5,3,[(test_cross_validation.py) The code does not ...,[{'File Path': '../../data/raw/openja/lightfm_...


In [11]:
score_var = consist_eval.evaluation_reports.groupby('model_name').agg({
    'score': ['var', 'count']
})
score_var

score      
                    var count
model_name                   
gpt-3.5-turbo  0.010204     2
gpt-4o         0.000000     2

In [12]:
import scipy
TAIL = 2
ALPHA = 0.05 #Or whatever you want your alpha to be.

f_score = score_var[('score', 'var')]['gpt-3.5-turbo'] / score_var[('score', 'var')]['gpt-4o'] # var(prev) / var(curr)
p_value = 1 - scipy.stats.f.cdf(
    f_score, 
    score_var[('score', 'count')]['gpt-3.5-turbo'] - 1, # degree of freedom (prev)
    score_var[('score', 'count')]['gpt-4o'] - 1         # degree of freedom (curr)
) 

print(f"p-value: {p_value}")
print()

print(f"{TAIL}-tail test:")
if p_value < ALPHA / 2 or p_value > 1 - ALPHA/2:
    print("  Successfully reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")
else:
    print("  Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")

/var/folders/vd/r3dvzdx10pxf47gvdqf81r9h0000gn/T/ipykernel_24102/1426530661.py:5: RuntimeWarning: divide by zero encountered in scalar divide
  f_score = score_var[('score', 'var')]['gpt-3.5-turbo'] / score_var[('score', 'var')]['gpt-4o'] # var(prev) / var(curr)


p-value: 0.0

2-tail test:
  Successfully reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))
